## 폐암환자 생존여부 머신러닝(SVM)으로 예측하기  - 4주차 온라인 4강 소스코드 참조

### 1. 사용 패키지와 모듈 임포트

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split #학습데이터와 테스트데이터 분할
from sklearn import svm   #SVM 모델
from sklearn import metrics  #정확도 asfnlslggl/ajg;jg;tjgjwjogwg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2. 데이터 가져오기

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_4주차_5강_6강_실습파일_오프라인/thoracic_surgery.csv')
#일부 데이터만 가져오기
#data_df = df[0:행개수] 
data_df = df
data_df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,Class
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0


In [ ]:
#각 행은 환자 한명 한명의 기록
#A1~A17 : 종양유형, 폐활량, 호흡곤란여부, 고통정도, 기침, 흡연, 천식여부 등의 수술 전 환자 상태
#         1은 해당사항 있음, 0은 해당 사항없음
#Class : 수술 후 생존 결과 (생존은 1, 사망은 0)

In [ ]:
#데이터의 행,열 개수 확인
data_df.shape  

(470, 18)

### 3. 데이터셋 나누기

In [ ]:
train, test = train_test_split(data_df, test_size= 0.2) 


In [ ]:
print("train data",train.shape)
print("test data",test.shape)

train data (376, 18)
test data (94, 18)


### 4. 학습에 사용할 변수(특징, Feature) 선택하기

In [ ]:
data_df.info()    #데이터 타입, 각 아이템 개수, 누락데이터(NaN(Not a Number or Not Available) 등 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      470 non-null    int64  
 1   A2      470 non-null    int64  
 2   A3      470 non-null    float64
 3   A4      470 non-null    float64
 4   A5      470 non-null    int64  
 5   A6      470 non-null    int64  
 6   A7      470 non-null    int64  
 7   A8      470 non-null    int64  
 8   A9      470 non-null    int64  
 9   A10     470 non-null    int64  
 10  A11     470 non-null    int64  
 11  A12     470 non-null    int64  
 12  A13     470 non-null    int64  
 13  A14     470 non-null    int64  
 14  A15     470 non-null    int64  
 15  A16     470 non-null    int64  
 16  A17     470 non-null    int64  
 17  Class   470 non-null    int64  
dtypes: float64(2), int64(16)
memory usage: 66.2 KB


In [ ]:
data_df.corr()   # 상관계수 보기

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,Class
A1,1.000000,-0.007878,-0.034985,0.029563,-0.021449,0.052724,0.027238,-0.061885,0.008845,0.027271,0.047079,-0.009229,-0.016382,-0.015761,-0.038681,-0.042400,-0.005826,-0.074924
A2,-0.007878,1.000000,0.121263,0.058780,-0.055900,0.047900,-0.062958,-0.047142,-0.075850,-0.019562,-0.135050,0.029753,-0.008675,-0.017461,-0.107427,-0.008675,0.076271,0.060444
A3,-0.034985,0.121263,1.000000,0.032975,-0.091094,0.019786,-0.095827,0.055829,-0.052770,-0.100242,0.034088,-0.115145,-0.009135,-0.035584,-0.012009,-0.060578,-0.290178,-0.046374
A4,0.029563,0.058780,0.032975,1.000000,-0.143155,0.161615,0.102979,0.260073,-0.099914,-0.086103,0.015504,-0.022251,-0.013617,-0.025088,-0.100853,-0.016509,-0.115900,-0.042841
A5,-0.021449,-0.055900,-0.091094,-0.143155,1.000000,0.092863,0.123296,0.092863,0.684647,0.418042,0.089751,0.025310,0.026788,0.023166,0.172289,-0.034330,0.214528,0.093200
A6,0.052724,0.047900,0.019786,0.161615,0.092863,1.000000,0.256225,0.067529,-0.024115,-0.072455,0.099942,0.022578,-0.017372,-0.034968,-0.077406,-0.017372,0.044789,0.057375
A7,0.027238,-0.062958,-0.095827,0.102979,0.123296,0.256225,1.000000,0.134386,0.081772,0.060393,0.059840,-0.001471,-0.026886,0.086156,-0.044942,-0.026886,0.086705,0.065785
A8,-0.061885,-0.047142,0.055829,0.260073,0.092863,0.067529,0.134386,1.000000,0.049843,-0.072455,0.075502,-0.042725,-0.017372,0.097572,-0.077406,-0.017372,-0.015331,0.105530
A9,0.008845,-0.075850,-0.052770,-0.099914,0.684647,-0.024115,0.081772,0.049843,1.000000,0.202245,0.145345,0.016551,0.044101,0.017815,0.200373,-0.026401,0.149589,0.088860
A10,0.027271,-0.019562,-0.100242,-0.086103,0.418042,-0.072455,0.060393,-0.072455,0.202245,1.000000,-0.036044,0.069522,0.058695,0.029726,0.118527,-0.029161,0.208003,0.086467


### 5. 학습용 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
train_data_df = train[['A8','A1','A2','A3','A4','A5','A6','A7','A9','A10','A11','A12','A15']]
train_data_df 

,A8,A1,A2,A3,A4,A5,A6,A7,A9,A10,A11,A12,A15
5,0,18,2,2.96,1.67,0,0,0,0,0,12,0,1
207,0,208,3,2.60,1.92,1,0,0,1,0,11,0,1
209,0,211,3,4.72,4.56,0,0,0,0,0,11,0,1
468,0,25,8,4.32,3.20,0,0,0,0,0,11,0,0
289,0,310,3,1.98,1.57,1,0,0,0,1,11,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,0,89,5,2.68,1.76,2,0,1,1,1,11,0,1
325,0,363,3,3.08,1.72,1,0,0,1,1,12,1,1
445,0,452,4,3.04,2.36,1,0,0,1,0,12,0,1
143,0,122,3,3.56,2.76,1,0,0,1,0,12,0,1


In [ ]:
train_label_df = train[['Class']]
train_label_df

,Class
5,0
207,0
209,0
468,1
289,0
...,...
453,0
325,1
445,0
143,0


In [ ]:
train_data = train_data_df.values           # 값부분만 가져오기(2차원 형태)
train_data

array([[  0.,  18.,   2., ...,  12.,   0.,   1.],
       [  0., 208.,   3., ...,  11.,   0.,   1.],
       [  0., 211.,   3., ...,  11.,   0.,   1.],
       ...,
       [  0., 452.,   4., ...,  12.,   0.,   1.],
       [  0., 122.,   3., ...,  12.,   0.,   1.],
       [  0., 215.,   3., ...,  11.,   0.,   1.]])

In [ ]:
train_label = train_label_df.values.ravel() # 값부분을 1차원으로 펴주기
train_label

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

### 6. 테스트 데이터셋 : 학습데이터와 레이블(정답) 나누기

In [ ]:
test_data_df = [['A8','A1','A2','A3','A4','A5','A6','A7','A9','A10','A11','A12','A15']]
test_data_df

[['A8',
  'A1',
  'A2',
  'A3',
  'A4',
  'A5',
  'A6',
  'A7',
  'A9',
  'A10',
  'A11',
  'A12',
  'A15']]

In [ ]:
test_label_df = [['Class']]
test_label_df

[['Class']]

In [ ]:
# 값부분만 가져오기(2차원 형태)
test_data = train_data_df.values     
test_data

array([[  0.,  18.,   2., ...,  12.,   0.,   1.],
       [  0., 208.,   3., ...,  11.,   0.,   1.],
       [  0., 211.,   3., ...,  11.,   0.,   1.],
       ...,
       [  0., 452.,   4., ...,  12.,   0.,   1.],
       [  0., 122.,   3., ...,  12.,   0.,   1.],
       [  0., 215.,   3., ...,  11.,   0.,   1.]])

In [ ]:
# 값부분을 1차원으로 펴주기
test_label = train_label_df.values.ravel() 
test_label

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

### 7. SVM 모델 학습하기

In [ ]:
#svm 객체 생성
clf = svm.SVC(C = 10, gamma = 1)    
#svm 모델로 학습데이터로 학습하기 fit()
clf.fit(train_data, train_label)


SVC(C=10, gamma=1)

### 8. 테스트 데이터로 예측하기

In [ ]:
#위에서 만든 clf(svm) 모델로 예측하기
pred_svm = clf.predict(test_data)
pred_svm  #svm이 예측한 생존 여부 값

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

### 9. 모델 예측 정확도 확인

In [ ]:
test_label #테스트 데이터에서 정답(생존 여부)


array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
ac_score = metrics.accuracy_score(test_label, pred_svm)
print('accuracy : ', ac_score)

accuracy :  1.0


In [ ]:
comparison = pd.DataFrame({'prediction_svm':pred_svm, 'ground_truth':test_label})
comparison

,prediction_svm,ground_truth
0,0,0
1,0,0
2,0,0
3,1,1
4,0,0
...,...,...
371,0,0
372,1,1
373,0,0
374,0,0


### * 주의  !!!! : 소스코드 중간 중간에 변경사항이 있다면 
### 맨 위에서부터 차례 차례 Shift + Enter를 모두 해주거나, 
### 상단 Kernel -> Restart & Run All로 전체적으로 다시 한번 실행해야 적용됨
### 고친 부분만 Shift + Enter 한다고 전체 소스코드에 반영되지 않음

## 1. 데이터를 증가시켰을 때 정확도는 얼마나 차이가 나는가?

##   200개일때 80, 전체에서 85%까지 증가한다.

## 2. 학습 피쳐(특징데이터, 변수)를 변경시켜 수행할 경우 정확도는 어떤 차이가 나는가?

##  A8,A1,A2,A15일 때 81.5%  A1, A2, A3, A4일 때는 정확도가 81.9%였다.

## 3. 비용(C)와 감마(gamma) 값을 변경하여 수행해보고 어떤 값이 가장 좋은 정확도를 갖는지 알아보자. 
### 비용이 0.1 , 1, 10 인 경우, 감마 값이 0.1, 1, 10 인 경우의 여러 조합으로 수행해 보자

## A. (C, gamma) 가 (10, 10) 일 떄 91.5%의 정확도로 가장 높았다.

## 4. 예측 정확도가 가장 높은 학습 모델의 데이터 수, 피쳐(변수) 및 파라메터의 값은 무엇인가?

## A.

In [ ]:
데이터 470개, 피쳐가 A8, A1,A2,A15이고 파라메터가 10,10일때

NameError: ignored